# E-GraphSAGE

In [6]:
from dgl import from_networkx
import sklearn
import torch.nn as nn
import torch as th
import torch.nn.functional as F
import dgl.function as fn
import networkx as nx
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import category_encoders as ce
import numpy as np
from sklearn.metrics import confusion_matrix

import os
from sklearn.utils import shuffle

from dgl.data.utils import save_graphs

#constante
size_embedding = 152
nb_batch = 5

#Data
nbclasses =  2

# Accuracy --------------------------------------------------------------------
def compute_accuracy(pred, labels):
    return (pred.argmax(1) == labels).float().mean().item()
# -----------------------------------------------------------------------------

# ------------------------------------------ Model Architecture -----------------------------------------------------------------

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
        super(SAGELayer, self).__init__()
        self.W_msg = nn.Linear(ndim_in + edims, ndim_out)
        self.W_apply = nn.Linear(ndim_in + ndim_out, ndim_out)
        self.activation = activation

    def message_func(self, edges):
        x = th.cat([edges.src['h'], edges.data['h']], 2)
        y = self.W_msg(x)
        return {'m': y}

    def forward(self, g_dgl, nfeats, efeats):
        with g_dgl.local_scope():
            g = g_dgl
            g.ndata['h'] = nfeats
            g.edata['h'] = efeats
            # Line 4 of algorithm 1 : update all because we are using a full neighborhood sampling and not a k-hop neigh sampling
            g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
            # Line 5 of algorithm 1
            g.ndata['h'] = F.relu(self.W_apply(th.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))
            return g.ndata['h']


class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation, dropout):
        super(SAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(SAGELayer(ndim_in, edim, size_embedding, activation))
        self.layers.append(SAGELayer(size_embedding, edim, size_embedding, activation)) ##
        self.layers.append(SAGELayer(size_embedding, edim, ndim_out, activation))
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, g, nfeats, efeats):
        
        for i, layer in enumerate(self.layers):
            if i != 0:
                nfeats = self.dropout(nfeats)
            nfeats = layer(g, nfeats, efeats)
            # Save edge_embeddings
            # nf = 'edge_embeddings'+str(i)+'.txt'
            # sourceFile = open(nf, 'w')
            # print(nfeats, file = sourceFile)
        return nfeats.sum(1)
        # Return a list of node features [[node1_feature1, node1_feature2, ...], [node2_feature1, node2_feature2, ...], ...]
    
class MLPPredictor(nn.Module):
    def __init__(self, in_features, out_classes):
        super().__init__()
        self.W = nn.Linear(in_features * 2, out_classes)

    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        v = th.cat([h_u, h_v], 1)
        # if(pr == True):
            # sourceFile = open(filename, 'w')
            # if pr:
                # print(v, file = sourceFile)
            # sourceFile.close()
        score = self.W(v)
        return {'score': score}

    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            # Update the features of the specified edges by the provided function
            # DGLGraph.apply_edges(func, edges='__ALL__', etype=None, inplace=False)
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']

class Model(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation, dropout):
        super().__init__()
        self.gnn = SAGE(ndim_in, ndim_out, edim, activation, dropout)
        self.pred = MLPPredictor(ndim_out, nbclasses)
    def forward(self, g, nfeats, efeats, eweight = None):
        if eweight != None:
            # apply eweight on the graph
            efe = []
            for i, x in enumerate(eweight):
                efe.append(list(th.Tensor.cpu(g.edata['h'][i][0]).detach().numpy() * th.Tensor.cpu(x).detach().numpy()))

            efe = th.FloatTensor(efe).cuda()
            efe = th.reshape(efe, (efe.shape[0], 1, efe.shape[1]))
            g.edata['h'] = efe = efe

        h = self.gnn(g, nfeats, efeats)
        # h = list of node features [[node1_feature1, node1_feature2, ...], [node2_feature1, node2_feature2, ...], ...]
        return self.pred(g, h)

# -------------------------------------------------------------------------------------------------------------------------------

## Graph + Ablation

In [29]:
# # --------------------------------------------------- MAIN -----------------------------------------------------------

import copy


# Model *******************************************************************************************
# G1.ndata['h'].shape[2] = sizeh = 76 dans ANIDS
# model1 = Model(G1.ndata['h'].shape[2], size_embedding, G1.ndata['h'].shape[2], F.relu, 0.2).cuda()
model1 = Model(76, size_embedding, 76, F.relu, 0.2).cuda()
opt = th.optim.Adam(model1.parameters())

model1_ab = Model(76, size_embedding, 76, F.relu, 0.2).cuda()
opt_ab = th.optim.Adam(model1_ab.parameters())



path, dirs, files = next(os.walk("/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/input/Dataset/GlobalDataset/Splitted/"))
file_count = len(files)


for nb_files in range(file_count):
    data1 = pd.read_csv(f'{path}{files[nb_files]}', encoding="ISO-8859–1", dtype = str)

    print(f'{files[nb_files]} ++++++++++++++++++++++++++++++++++++++++++++++')
    print("nb total instances in the file : ", len(data1.values))

    print("++++++++++++++++++++++++++++ Train ++++++++++++++++++++++++++++++++")
    
    # Delete two columns (U and V in the excel)
    cols = list(set(list(data1.columns )) - set(list(['Flow Bytes/s',' Flow Packets/s'])) )
    data1 = data1[cols]

    # Mise en forme des noeuds
    data1[' Source IP'] = data1[' Source IP'].apply(str)
    data1[' Source Port'] = data1[' Source Port'].apply(str)
    data1[' Destination IP'] = data1[' Destination IP'].apply(str)
    data1[' Destination Port'] = data1[' Destination Port'].apply(str)
    data1[' Source IP'] = data1[' Source IP'] + ':' + data1[' Source Port']
    data1[' Destination IP'] = data1[' Destination IP'] + ':' + data1[' Destination Port']

    data1.drop(columns=['Flow ID',' Source Port',' Destination Port',' Timestamp'], inplace=True)

    # -------------------- ????????????????????????????????????????? --------------------
    # simply do : nom = list(data1[' Label'].unique())
    nom = []
    nom = nom + [data1[' Label'].unique()[0]]
    for i in range(1, len(data1[' Label'].unique())):
        nom = nom + [data1[' Label'].unique()[i]]
    
    nom.insert(0, nom.pop(nom.index('BENIGN')))

    # Naming the two classes BENIGN {0} / Any Intrusion {1}
    data1[' Label'].replace(nom[0], 0,inplace = True)
    for i in range(1,len(data1[' Label'].unique())):
        data1[' Label'].replace(nom[i], 1,inplace = True)
    
    ##################### LABELS FREQ #######################################
    print()
    print("labels freq after changing labels to binary")
    counts = list(data1[' Label'].value_counts().to_dict().items())
    for j, x in enumerate(counts):
        x = list(x)
        x[1] = x[1] / len(data1)
        counts[j] = x
    print({f'{files[nb_files]}' : counts})
    ##############################################################################

    data1.rename(columns={" Label": "label"},inplace = True)
    label1 = data1.label
    data1.drop(columns=['label'],inplace = True)

    # ******** At this step data1 contains only the data without label column
    # ******** The label column is stored in the label variale 

    # split train and test
    data1 =  pd.concat([data1, label1], axis=1) # ??????? WHY ?
        

    # Split
    X1_train, X1_test, y1_train, y1_test = train_test_split(data1, label1, test_size=0.3, random_state=123, stratify= label1)

    
    # Create mini batches on the Train set
    X1_train = shuffle(X1_train)
    a = b = mean_macro_f1 = 0
    for batch in range(1, nb_batch + 1):
        print(f"+++++++++++++++++ Batch {batch} ++++++++++++++++")
        a = b
        b = int(len(X1_train) / nb_batch) * batch
        if batch == nb_batch :
            b = len(X1_train)
        # The batch :
        X1_train_batched = X1_train.iloc[a:b]
        # y1_train_batched = y1_train.iloc[a:b]
        y1_train_batched = X1_train_batched['label']        
        
        print("nb Train instances : ", len(X1_train_batched.values))

        # for non numerical attributes (categorical data)
        # Since we have a binary classification, the category values willl be replaced with the posterior probability (p(target = Ti | category = Cj))
        # TargetEncoding is also called MeanEncoding, cuz it simply replace each value with (target_i_count_on_category_j) / (total_occurences_of_category_j)
        encoder1 = ce.TargetEncoder(cols=[' Protocol',  'Fwd PSH Flags', ' Fwd URG Flags', ' Bwd PSH Flags', ' Bwd URG Flags'])
        encoder1.fit(X1_train_batched, y1_train_batched)
        X1_train_batched = encoder1.transform(X1_train_batched)

        # scaler (normalization)
        scaler1 = StandardScaler()

        # Manipulate flow content (all columns except : label, Source IP & Destination IP)
        cols_to_norm1 = list(set(list(X1_train_batched.iloc[:, :].columns )) - set(list(['label', ' Source IP', ' Destination IP'])) )
        X1_train_batched[cols_to_norm1] = scaler1.fit_transform(X1_train_batched[cols_to_norm1])

        ## Create the h attribute that will contain the content of our flows
        X1_train_batched['h'] = X1_train_batched[ cols_to_norm1 ].values.tolist()
        # size of the list containig the content of our flows
        sizeh = len(cols_to_norm1)


        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        # Before training the data :
        # We need to delete all the attributes (cols_to_norm1) to have the {Source IP, Destination IP, label, h} representation
        X1_train_batched.drop(columns = cols_to_norm1, inplace = True)
        
        # Edge index
        X1_train_batched['Edge_indx'] = list(range(len(X1_train_batched.values)))

        # Then we need to Swap {label, h} Columns to have the {Source IP, Destination IP, h, label} representation
        columns_titles = [' Source IP', ' Destination IP', 'h', 'label', 'Edge_indx']
        X1_train_batched = X1_train_batched.reindex(columns=columns_titles)
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

        # ------------------------------------------- Creating the Graph Representation -------------------------------------------------------------
        # Create our Multigraph
        G1 = nx.from_pandas_edgelist(X1_train_batched, " Source IP", " Destination IP", ['h','label', 'Edge_indx'], create_using=nx.MultiDiGraph())
        print("initial nx multigraph G1 : ", G1)

        G1 = from_networkx(G1,edge_attrs=['h','label', 'Edge_indx'] )
        print("G1.edata['h'] after converting it to a dgl graph : ", len(G1.edata['h']))

        # nodes data // G1.edata['h'].shape[1] : sizeh = number of attributes in a flow
        G1.ndata['h'] = th.ones(G1.num_nodes(), G1.edata['h'].shape[1])
        # edges data // we create a tensor bool array that will represent the train mask
        G1.edata['train_mask'] = th.ones(len(G1.edata['h']), dtype=th.bool)

        # Reshape both tensor lists to a single value in each element for both axis
        G1.ndata['h'] = th.reshape(G1.ndata['h'], (G1.ndata['h'].shape[0], 1, G1.ndata['h'].shape[1]))
        G1.edata['h'] = th.reshape(G1.edata['h'], (G1.edata['h'].shape[0], 1, G1.edata['h'].shape[1]))
        print("G1.edata['h'] after reshape : ", len(G1.edata['h']))
        # ------------------------------------------- --------------------------------- -------------------------------------------------------------

        

        
        # Ablation *************************************************************************
        X1_train_ab = copy.deepcopy(X1_train_batched)

        # print("data IP Addr before changing them : ")
        # print(X1_train_ab[[' Source IP', ' Destination IP']])

        X1_train_ab = X1_train_ab.drop(' Source IP', axis=1)
        X1_train_ab = X1_train_ab.drop(' Destination IP', axis=1)
        
        X1_train_ab[' Source IP'] = list(range(len(X1_train_ab.values)))
        X1_train_ab[' Destination IP'] = list(range(len(X1_train_ab.values), 2 * len(X1_train_ab.values)))

        print()
        # print("data IP Addr after changing them : ")
        # print(X1_train_ab[[' Source IP', ' Destination IP']])
        
        # ------------------------------------------- Creating the Graph Representation -------------------------------------------------------------
        # Create our Ablation Multigraph
        G1_ab = nx.from_pandas_edgelist(X1_train_ab, " Source IP", " Destination IP", ['h','label', 'Edge_indx'], create_using=nx.MultiDiGraph())
        print("initial nx multigraph G1_ab : ", G1_ab)

        G1_ab = from_networkx(G1_ab, edge_attrs=['h','label', 'Edge_indx'] )
        print("G1_ab.edata['h'] after converting it to a dgl graph : ", len(G1_ab.edata['h']))

        # nodes data // G1_ab.edata['h'].shape[1] : sizeh = number of attributes in a flow
        G1_ab.ndata['h'] = th.ones(G1_ab.num_nodes(), G1_ab.edata['h'].shape[1])
        # edges data // we create a tensor bool array that will represent the train mask
        G1_ab.edata['train_mask'] = th.ones(len(G1_ab.edata['h']), dtype=th.bool)

        # Reshape both tensor lists to a single value in each element for both axis
        G1_ab.ndata['h'] = th.reshape(G1_ab.ndata['h'], (G1_ab.ndata['h'].shape[0], 1, G1_ab.ndata['h'].shape[1]))
        G1_ab.edata['h'] = th.reshape(G1_ab.edata['h'], (G1_ab.edata['h'].shape[0], 1, G1_ab.edata['h'].shape[1]))
        print("G1_ab.edata['h'] after reshape : ", len(G1_ab.edata['h']))
        # ------------------------------------------- --------------------------------- -------------------------------------------------------------
        
        # ***********************************************************************************
        
        
        # ------------------------------------------- Model -----------------------------------------------------------------------------------------
        ## use of model
        from sklearn.utils import class_weight
        class_weights1 = class_weight.compute_class_weight(class_weight = 'balanced',
                                                        classes = np.unique(G1.edata['label'].cpu().numpy()),
                                                        y = G1.edata['label'].cpu().numpy())
        class_weights1 = th.FloatTensor(class_weights1).cuda()
        criterion1 = nn.CrossEntropyLoss(weight = class_weights1)
        G1 = G1.to('cuda:0')

        node_features1 = G1.ndata['h']
        edge_features1 = G1.edata['h']

        edge_label1 = G1.edata['label']
        train_mask1 = G1.edata['train_mask']


        # to print
        pr = True
        # True if you want to print the embedding vectors
        # the name of the file where the vectors are printed
        filename = './models/M1_weights.txt'
        
        print("\\\\\\\\\\\\\\\\ NORMAL \\\\\\\\\\\\\\\\\\")

        for epoch in range(1, 1000):
            pred = model1(G1, node_features1, edge_features1).cuda()
            loss = criterion1(pred[train_mask1], edge_label1[train_mask1])
            opt.zero_grad()
            loss.backward()
            opt.step()
            if epoch % 100 == 0:
                print('Training acc:', compute_accuracy(pred[train_mask1], edge_label1[train_mask1]), loss)

        pred1 = model1(G1, node_features1, edge_features1).cuda()
        pred1 = pred1.argmax(1)
        pred1 = th.Tensor.cpu(pred1).detach().numpy()
        edge_label1 = th.Tensor.cpu(edge_label1).detach().numpy()

        print('Train metrics :')
        print("Accuracy : ", sklearn.metrics.accuracy_score(edge_label1, pred1))
        print("Precision : ", sklearn.metrics.precision_score(edge_label1, pred1, labels = [0,1]))
        print("Recall : ", sklearn.metrics.recall_score(edge_label1, pred1, labels = [0,1]))
        print("f1_score : ", sklearn.metrics.f1_score(edge_label1, pred1, labels=[0,1]))
        
        

        
        # ------------------------------------------- AB Model -----------------------------------------------------------------------------------------
        ## use of model
        from sklearn.utils import class_weight
        class_weights1_ab = class_weight.compute_class_weight(class_weight = 'balanced',
                                                        classes = np.unique(G1_ab.edata['label'].cpu().numpy()),
                                                        y = G1_ab.edata['label'].cpu().numpy())
        class_weights1_ab = th.FloatTensor(class_weights1_ab).cuda()
        criterion1_ab = nn.CrossEntropyLoss(weight = class_weights1_ab)
        G1_ab = G1_ab.to('cuda:0')

        node_features1_ab = G1_ab.ndata['h']
        edge_features1_ab = G1_ab.edata['h']

        edge_label1_ab = G1_ab.edata['label']
        train_mask1_ab = G1_ab.edata['train_mask']
        
        print("\\\\\\\\\\\\\\\\ ABLATION \\\\\\\\\\\\\\\\\\")
        
        for epoch in range(1, 1000):
            pred_ab = model1_ab(G1_ab, node_features1_ab, edge_features1_ab).cuda()
            loss_ab = criterion1_ab(pred_ab[train_mask1], edge_label1_ab[train_mask1])
            opt_ab.zero_grad()
            loss_ab.backward()
            opt_ab.step()
            if epoch % 100 == 0:
                print('Training acc:', compute_accuracy(pred_ab[train_mask1], edge_label1_ab[train_mask1]), loss_ab)

        pred1_ab = model1_ab(G1_ab, node_features1_ab, edge_features1_ab).cuda()
        pred1_ab = pred1_ab.argmax(1)
        pred1_ab = th.Tensor.cpu(pred1_ab).detach().numpy()
        edge_label1_ab = th.Tensor.cpu(edge_label1_ab).detach().numpy()

        print('Train metrics :')
        print("Accuracy : ", sklearn.metrics.accuracy_score(edge_label1_ab, pred1_ab))
        print("Precision : ", sklearn.metrics.precision_score(edge_label1_ab, pred1_ab, labels = [0,1]))
        print("Recall : ", sklearn.metrics.recall_score(edge_label1_ab, pred1_ab, labels = [0,1]))
        print("f1_score : ", sklearn.metrics.f1_score(edge_label1_ab, pred1_ab, labels=[0,1]))

        



    # ------------------------------------------------ Test ---------------------------------------------------------------------
    print("++++++++++++++++++++++++++++ Test ++++++++++++++++++++++++++++++++")
    print("nb Test instances : ", len(X1_test.values))
    X1_test = encoder1.transform(X1_test)
    X1_test[cols_to_norm1] = scaler1.transform(X1_test[cols_to_norm1])

    # Save X1_test for XAI
    X1_test.to_csv(f'/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/input/Dataset/XAI/X_test{nb_files}.csv', sep=',', index = False)

    X1_test['h'] = X1_test[ cols_to_norm1 ].values.tolist()

    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # Before training the data :
    # We need to delete all the attributes (cols_to_norm1) to have the {Source IP, Destination IP, label, h} representation
    X1_test.drop(columns = cols_to_norm1, inplace = True)
    
    # Edge index
    X1_test['Edge_indx'] = list(range(len(X1_test.values)))

    # Then we need to Swap {label, h} Columns to have the {Source IP, Destination IP, h, label} representation
    columns_titles = [' Source IP', ' Destination IP', 'h', 'label', 'Edge_indx']
    X1_test = X1_test.reindex(columns=columns_titles)
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    
    # Graph Construction
    G1_test = nx.from_pandas_edgelist(X1_test, " Source IP", " Destination IP", ['h','label', 'Edge_indx'],create_using=nx.MultiDiGraph())
    # G1_test = G1_test.to_directed()
    G1_test = from_networkx(G1_test,edge_attrs=['h','label', 'Edge_indx'] )
    # actual1 = G1_test.edata.pop('label')
    actual1 = G1_test.edata['label']
    G1_test.ndata['feature'] = th.ones(G1_test.num_nodes(), G1.ndata['h'].shape[2])
    G1_test.ndata['feature'] = th.reshape(G1_test.ndata['feature'], (G1_test.ndata['feature'].shape[0], 1, G1_test.ndata['feature'].shape[1]))
    G1_test.edata['h'] = th.reshape(G1_test.edata['h'], (G1_test.edata['h'].shape[0], 1, G1_test.edata['h'].shape[1]))
    G1_test = G1_test.to('cuda:0')
    node_features_test1 = G1_test.ndata['feature']
    edge_features_test1 = G1_test.edata['h']
    
    
    
    # Ablation *************************************************************************
    X1_test_ab = copy.deepcopy(X1_test)

    # print("data IP Addr before changing them : ")
    # print(X1_test_ab[[' Source IP', ' Destination IP']])

    X1_test_ab = X1_test_ab.drop(' Source IP', axis=1)
    X1_test_ab = X1_test_ab.drop(' Destination IP', axis=1)

    X1_test_ab[' Source IP'] = list(range(len(X1_test_ab.values)))
    X1_test_ab[' Destination IP'] = list(range(len(X1_test_ab.values), 2 * len(X1_test_ab.values)))

    print()
    # print("data IP Addr after changing them : ")
    # print(X1_test_ab[[' Source IP', ' Destination IP']])
    
    # Graph Construction
    G1_test_ab = nx.from_pandas_edgelist(X1_test_ab, " Source IP", " Destination IP", ['h','label', 'Edge_indx'],create_using=nx.MultiDiGraph())
    G1_test_ab = from_networkx(G1_test_ab, edge_attrs=['h','label', 'Edge_indx'] )
    actual1_ab = G1_test_ab.edata['label']
    G1_test_ab.ndata['feature'] = th.ones(G1_test_ab.num_nodes(), G1_ab.ndata['h'].shape[2])
    G1_test_ab.ndata['feature'] = th.reshape(G1_test_ab.ndata['feature'], (G1_test_ab.ndata['feature'].shape[0], 1, G1_test_ab.ndata['feature'].shape[1]))
    G1_test_ab.edata['h'] = th.reshape(G1_test_ab.edata['h'], (G1_test_ab.edata['h'].shape[0], 1, G1_test_ab.edata['h'].shape[1]))
    G1_test_ab = G1_test_ab.to('cuda:0')
    node_features_test1_ab = G1_test_ab.ndata['feature']
    edge_features_test1_ab = G1_test_ab.edata['h']
    
    # ***********************************************************************************

    print("\\\\\\\\\\\\\\\\ NORMAL \\\\\\\\\\\\\\\\\\")
    print("nb instances : ", len(X1_test.values))

    test_pred1 = model1(G1_test, node_features_test1, edge_features_test1).cuda()
    test_pred1 = test_pred1.argmax(1)
    test_pred1 = th.Tensor.cpu(test_pred1).detach().numpy()

    print('Metrics : ')
    print("Accuracy : ", sklearn.metrics.accuracy_score(actual1, test_pred1))
    print("Precision : ", sklearn.metrics.precision_score(actual1, test_pred1, labels = [0,1]))
    print("Recall : ", sklearn.metrics.recall_score(actual1, test_pred1, labels = [0,1]))
    print("f1_score : ", sklearn.metrics.f1_score(actual1, test_pred1, labels = [0,1]))
    
    
    print("\\\\\\\\\\\\\\\\ ABLATION \\\\\\\\\\\\\\\\\\")
    print("nb instances : ", len(X1_test_ab.values))

    test_pred1_ab = model1_ab(G1_test_ab, node_features_test1_ab, edge_features_test1_ab).cuda()
    test_pred1_ab = test_pred1_ab.argmax(1)
    test_pred1_ab = th.Tensor.cpu(test_pred1_ab).detach().numpy()

    print('Metrics : ')
    print("Accuracy : ", sklearn.metrics.accuracy_score(actual1_ab, test_pred1_ab))
    print("Precision : ", sklearn.metrics.precision_score(actual1_ab, test_pred1_ab, labels = [0,1]))
    print("Recall : ", sklearn.metrics.recall_score(actual1_ab, test_pred1_ab, labels = [0,1]))
    print("f1_score : ", sklearn.metrics.f1_score(actual1_ab, test_pred1_ab, labels = [0,1]))

    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

CIC-IDS-2017-Dataset4.csv ++++++++++++++++++++++++++++++++++++++++++++++
nb total instances in the file :  460167
++++++++++++++++++++++++++++ Train ++++++++++++++++++++++++++++++++

labels freq after changing labels to binary
{'CIC-IDS-2017-Dataset4.csv': [[0, 0.7582073464633492], [1, 0.24179265353665083]]}
+++++++++++++++++ Batch 1 ++++++++++++++++
nb Train instances :  64423
initial nx multigraph G1 :  MultiDiGraph with 58978 nodes and 64423 edges
G1.edata['h'] after converting it to a dgl graph :  64423
G1.edata['h'] after reshape :  64423

initial nx multigraph G1_ab :  MultiDiGraph with 128846 nodes and 64423 edges
G1_ab.edata['h'] after converting it to a dgl graph :  64423
G1_ab.edata['h'] after reshape :  64423
\\\\\\\\ NORMAL \\\\\\\\\
Training acc: 0.9852691888809204 tensor(0.0346, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9869921803474426 tensor(0.0261, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9873492121696472 tensor(0.0245, device='cu

Training acc: 0.9914160966873169 tensor(0.0180, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9915247559547424 tensor(0.0177, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9914937019348145 tensor(0.0176, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9915247559547424 tensor(0.0174, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.991540253162384 tensor(0.0174, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9914781451225281 tensor(0.0180, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9915247559547424 tensor(0.0174, device='cuda:0', grad_fn=<NllLossBackward0>)
Train metrics :
Accuracy :  0.9915402884063145
Precision :  0.9660826032540676
Recall :  0.9998057120652808
f1_score :  0.9826549123197861
\\\\\\\\ ABLATION \\\\\\\\\
Training acc: 0.9584309458732605 tensor(0.0891, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.971314549446106 tensor(0.0673, device='cuda:0', grad_fn=<NllLossBackward0>

Training acc: 0.9901121854782104 tensor(0.0190, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9911832213401794 tensor(0.0192, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9914626479148865 tensor(0.0185, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9915868043899536 tensor(0.0182, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9914160966873169 tensor(0.0182, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9909814596176147 tensor(0.0185, device='cuda:0', grad_fn=<NllLossBackward0>)
Train metrics :
Accuracy :  0.9915868556260963
Precision :  0.9675101530771634
Recall :  0.998581285870897
f1_score :  0.9828002030972328
\\\\\\\\ ABLATION \\\\\\\\\
Training acc: 0.9717646837234497 tensor(0.0624, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9726805090904236 tensor(0.0578, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9713456034660339 tensor(0.0592, device='cuda:0', grad_fn=<NllLossBackward0

Training acc: 0.975878119468689 tensor(0.0475, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9759712815284729 tensor(0.0470, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9771044254302979 tensor(0.0461, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.977570116519928 tensor(0.0463, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9774614572525024 tensor(0.0456, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9769181609153748 tensor(0.0449, device='cuda:0', grad_fn=<NllLossBackward0>)
Train metrics :
Accuracy :  0.97833072039489
Precision :  0.9274236903410787
Recall :  0.9884764754568027
f1_score :  0.9569773175542408
++++++++++++++++++++++++++++ Test ++++++++++++++++++++++++++++++++
nb Test instances :  138050

\\\\\\\\ NORMAL \\\\\\\\\
nb instances :  138050
Metrics : 
Accuracy :  0.9919087287214777
Precision :  0.9690592981743245
Recall :  0.9984747891620313
f1_score :  0.9835471564714027
\\\\\\\\ ABLATION \\\\\\\\\


Training acc: 0.9625443816184998 tensor(0.0735, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9618614315986633 tensor(0.0712, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9602315425872803 tensor(0.0772, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9643760323524475 tensor(0.0701, device='cuda:0', grad_fn=<NllLossBackward0>)
Train metrics :
Accuracy :  0.9626220449218447
Precision :  0.8735189069675152
Recall :  0.9886429258902791
f1_score :  0.9275222730556225
+++++++++++++++++ Batch 4 ++++++++++++++++
nb Train instances :  64423
initial nx multigraph G1 :  MultiDiGraph with 58969 nodes and 64423 edges
G1.edata['h'] after converting it to a dgl graph :  64423
G1.edata['h'] after reshape :  64423

initial nx multigraph G1_ab :  MultiDiGraph with 128846 nodes and 64423 edges
G1_ab.edata['h'] after converting it to a dgl graph :  64423
G1_ab.edata['h'] after reshape :  64423
\\\\\\\\ NORMAL \\\\\\\\\
Training acc: 0.990531325340271 tensor(0.01

Training acc: 0.9832512736320496 tensor(0.0347, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9832823276519775 tensor(0.0346, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9834375381469727 tensor(0.0341, device='cuda:0', grad_fn=<NllLossBackward0>)
Train metrics :
Accuracy :  0.9835928162302283
Precision :  0.9417954020192191
Recall :  0.9935835739493102
f1_score :  0.9669965966216005
+++++++++++++++++ Batch 2 ++++++++++++++++
nb Train instances :  64423
initial nx multigraph G1 :  MultiDiGraph with 58979 nodes and 64423 edges
G1.edata['h'] after converting it to a dgl graph :  64423
G1.edata['h'] after reshape :  64423

initial nx multigraph G1_ab :  MultiDiGraph with 128846 nodes and 64423 edges
G1_ab.edata['h'] after converting it to a dgl graph :  64423
G1_ab.edata['h'] after reshape :  64423
\\\\\\\\ NORMAL \\\\\\\\\
Training acc: 0.9901742935180664 tensor(0.0278, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9913074374198914 tensor(0.0

Training acc: 0.9906401634216309 tensor(0.0187, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9902831315994263 tensor(0.0186, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9903297424316406 tensor(0.0185, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9905780553817749 tensor(0.0184, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9909661412239075 tensor(0.0179, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9904693961143494 tensor(0.0181, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.990826427936554 tensor(0.0178, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9914628267288208 tensor(0.0175, device='cuda:0', grad_fn=<NllLossBackward0>)
Train metrics :
Accuracy :  0.9919129516950205
Precision :  0.9684484038604306
Recall :  0.9992977976380466
f1_score :  0.9836312796506331
\\\\\\\\ ABLATION \\\\\\\\\
Training acc: 0.9669067859649658 tensor(0.0690, device='cuda:0', grad_fn=<NllLossBackward0

Training acc: 0.9903138875961304 tensor(0.0194, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9902052283287048 tensor(0.0191, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9914315342903137 tensor(0.0184, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9905933141708374 tensor(0.0187, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9904536008834839 tensor(0.0187, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9913073778152466 tensor(0.0183, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9904846549034119 tensor(0.0182, device='cuda:0', grad_fn=<NllLossBackward0>)
Train metrics :
Accuracy :  0.9909192511874826
Precision :  0.964287913305831
Recall :  0.9996808374824461
f1_score :  0.9816654652584074
\\\\\\\\ ABLATION \\\\\\\\\
Training acc: 0.9601068496704102 tensor(0.0832, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9613641500473022 tensor(0.0770, device='cuda:0', grad_fn=<NllLossBackward0

### ACC + F1

In [30]:
print("\\\\\\\\\\\\\\\\ NORMAL \\\\\\\\\\\\\\\\\\")
print("nb instances : ", len(X1_test.values))

test_pred1 = model1(G1_test, node_features_test1, edge_features_test1).cuda()
test_pred1 = test_pred1.argmax(1)
test_pred1 = th.Tensor.cpu(test_pred1).detach().numpy()

print('Metrics : ')
print("Accuracy : ", sklearn.metrics.accuracy_score(actual1, test_pred1))
print("Precision : ", sklearn.metrics.precision_score(actual1, test_pred1, labels = [0,1]))
print("Recall : ", sklearn.metrics.recall_score(actual1, test_pred1, labels = [0,1]))
print("f1_score : ", sklearn.metrics.f1_score(actual1, test_pred1, labels = [0,1]))


print("\\\\\\\\\\\\\\\\ ABLATION \\\\\\\\\\\\\\\\\\")
print("nb instances : ", len(X1_test_ab.values))

test_pred1_ab = model1_ab(G1_test_ab, node_features_test1_ab, edge_features_test1_ab).cuda()
test_pred1_ab = test_pred1_ab.argmax(1)
test_pred1_ab = th.Tensor.cpu(test_pred1_ab).detach().numpy()

print('Metrics : ')
print("Accuracy : ", sklearn.metrics.accuracy_score(actual1_ab, test_pred1_ab))
print("Precision : ", sklearn.metrics.precision_score(actual1_ab, test_pred1_ab, labels = [0,1]))
print("Recall : ", sklearn.metrics.recall_score(actual1_ab, test_pred1_ab, labels = [0,1]))
print("f1_score : ", sklearn.metrics.f1_score(actual1_ab, test_pred1_ab, labels = [0,1]))

\\\\\\\\ NORMAL \\\\\\\\\
nb instances :  138050
Metrics : 
Accuracy :  0.9911119159724737
Precision :  0.9679029543212027
Recall :  0.9963851462372658
f1_score :  0.9819375542830224
\\\\\\\\ ABLATION \\\\\\\\\
nb instances :  138050
Metrics : 
Accuracy :  0.9753350235421948
Precision :  0.9160163816481266
Recall :  0.9889463149403998
f1_score :  0.9510853169757654


### Save X_Test + Models + Graphs + Predictions

In [31]:
X1_test.to_csv(f'/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Global_X_Test.csv', sep=',', index = False)

# Models
th.save(model1.state_dict(), '/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Models/GNN.pt')
th.save(model1_ab.state_dict(), '/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Models/GNN_ab.pt')

# Graphs
from dgl.data.utils import save_graphs
save_graphs("/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Models/G1_test.bin", [G1_test])
save_graphs("/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Models/G1_test_ab.bin", [G1_test_ab])

np.savetxt('/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Models/test_pred1.txt', test_pred1, fmt='%d')
np.savetxt('/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Models/test_pred1_ab.txt', test_pred1_ab, fmt='%d')

### Load graphs and predictions

In [7]:
# loading Graphs and Predictions
from dgl.data.utils import load_graphs
import numpy as np

Test_Graph = load_graphs("/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Models/G1_test.bin")
Test_Graph = Test_Graph[0][0]
Test_Graph_ab = load_graphs("/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Models/G1_test_ab.bin")
Test_Graph_ab = Test_Graph_ab[0][0]

Test_pred_Graph = np.loadtxt('/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Models/test_pred1.txt', dtype=int)
Test_pred_Graph_ab = np.loadtxt('/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Models/test_pred1_ab.txt', dtype=int)


actual1 = Test_Graph.edata['label']
actual1_ab = Test_Graph_ab.edata['label']


# Test the loading
print(actual1)
print(actual1_ab)
aa = actual1 == actual1_ab
aa = aa.numpy()
print(np.where(aa == False))

print(Test_pred_Graph)
print(Test_pred_Graph_ab)
aa = Test_pred_Graph == Test_pred_Graph_ab

print(type(Test_pred_Graph))
print(type(Test_pred_Graph_ab))
print(type(aa))

print(aa)
print(np.where(aa == False))

tensor([1, 1, 0,  ..., 0, 0, 0])
tensor([1, 0, 0,  ..., 0, 0, 0])
(array([     1,      9,     14, ..., 138035, 138036, 138040]),)
[1 1 0 ... 0 0 0]
[1 0 0 ... 0 1 0]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[ True False  True ...  True False  True]
(array([     1,      9,     14, ..., 138038, 138040, 138048]),)


### Models testing on the graphs

In [10]:
model1_test = Model(76, size_embedding, 76, F.relu, 0.2).cuda()
model1_test.load_state_dict(th.load('/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Models/GNN.pt'))
model1_test.eval()

model1_test_ab = Model(76, size_embedding, 76, F.relu, 0.2).cuda()
model1_test_ab.load_state_dict(th.load('/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Models/GNN_ab.pt'))
model1_test_ab.eval()


print("\\\\\\\\\\\\\\\\ NORMAL \\\\\\\\\\\\\\\\\\")

Test_Graph1 = Test_Graph.to('cuda:0')
node_features_test1 = Test_Graph1.ndata['feature']
edge_features_test1 = Test_Graph1.edata['h']

test_pred1 = model1_test(Test_Graph1, node_features_test1, edge_features_test1).cuda()
test_pred1 = test_pred1.argmax(1)
test_pred1 = th.Tensor.cpu(test_pred1).detach().numpy()

print('Metrics : ')
print("Accuracy : ", sklearn.metrics.accuracy_score(actual1, test_pred1))
print("Precision : ", sklearn.metrics.precision_score(actual1, test_pred1, labels = [0,1]))
print("Recall : ", sklearn.metrics.recall_score(actual1, test_pred1, labels = [0,1]))
print("f1_score : ", sklearn.metrics.f1_score(actual1, test_pred1, labels = [0,1]))


print("\\\\\\\\\\\\\\\\ ABLATION \\\\\\\\\\\\\\\\\\")

Test_Graph_ab1 = Test_Graph_ab.to('cuda:0')
node_features_test1_ab = Test_Graph_ab1.ndata['feature']
edge_features_test1_ab = Test_Graph_ab1.edata['h']

test_pred1_ab = model1_test_ab(Test_Graph_ab1, node_features_test1_ab, edge_features_test1_ab).cuda()
test_pred1_ab = test_pred1_ab.argmax(1)
test_pred1_ab = th.Tensor.cpu(test_pred1_ab).detach().numpy()

print('Metrics : ')
print("Accuracy : ", sklearn.metrics.accuracy_score(actual1_ab, test_pred1_ab))
print("Precision : ", sklearn.metrics.precision_score(actual1_ab, test_pred1_ab, labels = [0,1]))
print("Recall : ", sklearn.metrics.recall_score(actual1_ab, test_pred1_ab, labels = [0,1]))
print("f1_score : ", sklearn.metrics.f1_score(actual1_ab, test_pred1_ab, labels = [0,1]))

\\\\\\\\ NORMAL \\\\\\\\\
Metrics : 
Accuracy :  0.9911626222383194
Precision :  0.9677737490935461
Recall :  0.9967436441310907
f1_score :  0.9820450933066464
\\\\\\\\ ABLATION \\\\\\\\\
Metrics : 
Accuracy :  0.9749873234335386
Precision :  0.9135441921974873
Recall :  0.9905894302870971
f1_score :  0.9505081053189812


### Extract the 3% of difference

In [26]:
print(Test_Graph.edata['Edge_indx'])
print(Test_Graph_ab.edata['Edge_indx'])

tensor([     0,  71207,     22,  ..., 138044, 138045, 138046])
tensor([     0,      1,      2,  ..., 138047, 138048, 138049])


In [27]:
actual1_np = actual1.numpy()
aa = actual1_np == Test_pred_Graph
wrong_preds = np.where(aa == False)[0]
print(wrong_preds)

actual1_ab_np = actual1_ab.numpy()
bb = actual1_ab_np == Test_pred_Graph_ab
wrong_preds_ab = np.where(bb == False)[0]
print(wrong_preds_ab)

[  3495   3825   4535 ... 137915 137984 137995]
[   107    167    274 ... 138031 138038 138048]


In [36]:
wpred = Test_Graph.edata['Edge_indx'][wrong_preds]
wpred_ab = Test_Graph_ab.edata['Edge_indx'][wrong_preds_ab]

print(wpred)
print(wpred_ab)

edges_to_explain = set(wpred_ab.numpy()) - set(wpred.numpy())

print(len(edges_to_explain))
print(edges_to_explain)

tensor([    76,  87306,  79223,  ..., 137736, 137871, 137893])
tensor([   107,    167,    274,  ..., 138031, 138038, 138048])
3124
{65536, 49153, 8194, 24579, 122884, 16391, 73741, 81936, 49170, 49171, 49173, 65560, 106525, 24609, 98343, 90152, 131111, 122923, 90158, 81968, 98355, 114743, 73786, 65595, 106556, 98369, 16453, 90181, 98376, 106568, 16459, 41040, 49233, 65620, 90201, 8283, 122974, 41056, 57440, 32868, 24678, 41063, 107, 57451, 49262, 24689, 73842, 106623, 98432, 24705, 57471, 123014, 41100, 8334, 123028, 49301, 131220, 98456, 106649, 8346, 82076, 24736, 114850, 98469, 106662, 167, 16552, 98477, 106670, 123055, 90288, 41138, 131250, 114870, 16569, 106681, 65725, 49347, 16580, 49348, 65734, 106694, 114891, 24787, 106715, 49376, 24804, 90341, 106724, 41191, 106725, 114916, 90356, 41207, 123131, 24829, 57597, 98560, 16641, 33025, 33027, 49415, 49416, 16649, 98567, 8459, 73999, 274, 123156, 57625, 82202, 33056, 65830, 33063, 65834, 82218, 24881, 16700, 115006, 106816, 57665, 13

In [72]:
indx_to_explain = []

for x in edges_to_explain:
    indx_to_explain.append((Test_Graph.edata['Edge_indx'] == x).nonzero(as_tuple=True)[0].item())

# print(indx_to_explain)

df_indx = pd.DataFrame(columns = ['Edge_indx', 'label'])

for x in indx_to_explain:
    df_indx.loc[-1] = [x, Test_Graph.edata['label'][x].item()]  # adding a row
    df_indx.index = df_indx.index + 1  # shifting index

    
df_indx = df_indx.sort_values('label')
print(df_indx)

print('nb attacks :', len(df_indx.loc[df_indx['label'] == 1]))
print('nb benign :', len(df_indx.loc[df_indx['label'] == 0]))

df_indx.to_csv(f'/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Models/Edges_to_explain.csv', sep=',', index = False)

      Edge_indx  label
3123      98607      0
1106     102377      0
1105       4448      0
1104      39811      0
1103      23951      0
...         ...    ...
348      127231      1
2516      42725      1
351       84483      1
334        7892      1
1657      85275      1

[3124 rows x 2 columns]
nb attacks : 322
nb benign : 2802


### GNN-Edge-Explainer (Local Explanation for each one of the 3% edges)

In [106]:
# Local Explanation ***********************************************************************
from math import sqrt
from tqdm import tqdm
from dgl import EID, NID, khop_out_subgraph

import torch.nn as nn
import torch as th


# init mask
def init_masks(graph, efeat):
    # efeat.size() = torch.Size([nb_edges, 1, 76])
    efeat_size = efeat.size()[2]
    num_edges = graph.num_edges()
    num_nodes = graph.num_nodes()

    device = efeat.device

    std = 0.1
    # feat_mask = [[f1, f2, .... fn]] / n = nb_features
    efeat_mask = nn.Parameter(th.randn(1, efeat_size, device=device) * std)

    std = nn.init.calculate_gain("relu") * sqrt(2.0 / (2 * num_nodes))
    # edge_mask = [e1, e2, .... em] / m = nb_edges
    edge_mask = nn.Parameter(th.randn(num_edges, device=device) * std)

    # print("efeat_mask : ", efeat_mask)
    # print("edge_mask : ", edge_mask)

    return efeat_mask, edge_mask


# Regularization loss
def loss_regularize(loss, feat_mask, edge_mask):
    # epsilon for numerical stability
    eps = 1e-15
    # From self GNNExplainer self
    alpha1 = 0.005,
    alpha2 = 1.0
    beta1 = 1.0
    beta2 = 0.1

    edge_mask = edge_mask.sigmoid()
    # Edge mask sparsity regularization
    loss = loss + th.from_numpy(alpha1 * th.Tensor.cpu(th.sum(edge_mask)).detach().numpy()).cuda()
    # Edge mask entropy regularization
    ent = -edge_mask * th.log(edge_mask + eps) - (
        1 - edge_mask
    ) * th.log(1 - edge_mask + eps)
    loss = loss + alpha2 * ent.mean()

    feat_mask = feat_mask.sigmoid()
    # Feature mask sparsity regularization
    loss = loss + beta1 * th.mean(feat_mask)
    # Feature mask entropy regularization
    ent = -feat_mask * th.log(feat_mask + eps) - (
        1 - feat_mask
    ) * th.log(1 - feat_mask + eps)
    loss = loss + beta2 * ent.mean()

    return loss



# Edge mask
def explain_edges(model, edge_id, graph, node_feat, edge_feat, **kwargs):
    model = model.to(graph.device)
    model.eval()

    #print(graph.edges())

    # Extract source node-centered k-hop subgraph from the edge_id and its associated node and edge features.
    num_hops = 3
    source_node = th.Tensor.cpu(graph.edges()[0][edge_id]).detach().numpy()
    #print("source_node : ", source_node)
    edge_h = graph.edata['h'][edge_id]
    sg, inverse_indices = khop_out_subgraph(graph, source_node, num_hops)
    #print("new_node_indice : ", inverse_indices)

    #print(sg.edges())
    #print(edge_h)
    #print(sg.edata['h'])

    for indx, nd_id in enumerate(sg.edges()[0]):
        if inverse_indices == nd_id :
            if (sg.edata['h'][indx][0] == edge_h[0]).all() :
                # print("edge index is : ", indx)
                edge_indice = indx
                break
    
    #print("new_edge_indice : ", edge_indice)

    # EID = NID = _ID
    # tensor([0, 1, 2, 4]) : nodes and edges ids
    sg_edges = sg.edata[EID].long()
    sg_nodes = sg.ndata[NID].long()

    #print("+++++++++++++++++++++++")
    #print("sg : ", sg)
    #print("sg_edges : ", sg_edges) # edges ids in graph.edges()
    #print("sg_nodes : ", sg_nodes) # nodes ids in graph.nodes()

    #print()
    edge_feat = edge_feat[sg_edges]
    node_feat = node_feat[sg_nodes]

    #print("edge_feat : ", edge_feat)
    #print("node_feat : ", node_feat)
    #print("+++++++++++++++++++++++")
    
    
    # Get the initial prediction.
    #print("Get the initial prediction :")
    with th.no_grad():
        # logits = model(g = sg, nfeats = node_feat, efeats = edge_feat, **kwargs)
        logits = model(g = sg, nfeats = node_feat, efeats = edge_feat)
        pred_label = logits.argmax(dim=-1)
        # pred_label1 = logits.argmax(1)

    #print("pred_label : ", pred_label)
    # print(pred_label1)

    #
    efeat_mask, edge_mask = init_masks(sg, edge_feat)

    params = [edge_mask]
    optimizer = th.optim.Adam(params, lr = 0.01)

    # num_epochs = 300
    #print("***********************************")
    #print("initial masks : ")
    #print("efeat_mask : ", efeat_mask)
    #print("edge_mask : ", edge_mask)
    #print("***********************************")
    
    
    from sklearn.utils import class_weight
    # class_weights2 = class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(sg.edata['label'].cpu().numpy()), y = sg.edata['label'].cpu().numpy())
    # class_weights2 = class_weight.compute_class_weight(class_weight = 'balanced', classes = np.array([0, 1]), y = sg.edata['label'].cpu().numpy())
    # class_weights2 = th.FloatTensor(class_weights2).cuda()
    # criterion2 = nn.CrossEntropyLoss(weight = class_weights2)
    criterion2 = nn.CrossEntropyLoss()
    train_mask2 = th.ones(len(sg.edata['h']), dtype=th.bool)
    import datetime
    
    #print(f'explanation starts at {datetime.datetime.now()}')
    #print("nb edges : ", sg.num_edges())
    #print("nb nodes : ", sg.num_nodes())
    
    
    for epoch in range(1, 300):
        optimizer.zero_grad()
        # Edge mask
        logits = model(g = sg, nfeats = node_feat, efeats = edge_feat, eweight=edge_mask.sigmoid()).cuda()
        # logits = model(g = sg, nfeats = node_feat, efeats = h)
        # pred_label = tensor([0, 0, 0,  ..., 0, 1, 0], device='cuda:0')
        # logits = tensor([[ 0.0059,  0.0517], [-0.0075,  0.0101], ..., device='cuda:0', grad_fn=<IndexBackward0>)
        # log_probs = logits.log_softmax(dim=-1)
        # loss = -log_probs[edge_indice, pred_label[edge_indice]]
        loss11 = criterion2(logits[train_mask2], pred_label[train_mask2])
        loss = loss_regularize(loss11, efeat_mask, edge_mask)
        # loss = loss_regularize(loss, efeat_mask, edge_mask)
        
        #if epoch % 100 == 0:
            #print("+++++++++++++++")
            #print(f'epoch number {epoch}, CrossEntropy_Loss = {loss11}, final_loss = {loss}, time = {datetime.datetime.now()}')
            #print("edge_mask : ", edge_mask.detach().sigmoid())
        
        loss.backward()
        optimizer.step()

    #print("final results before sigmoid : ")
    #print("edge_mask : ", edge_mask)
    #print("***********************************")

    edge_mask = edge_mask.detach().sigmoid()

    return edge_indice, sg, edge_mask, loss.item()

In [112]:
Test_Graph1 = Test_Graph.to('cuda:0')
node_features_test1 = Test_Graph1.ndata['feature']
edge_features_test1 = Test_Graph1.edata['h']

edge_indice, sub_graph, edge_mask, loss = explain_edges(model1_test, 98607, Test_Graph1, node_features_test1, edge_features_test1)

print("final results : ")
print("edge_mask : ", edge_mask)
print("sub_graph : ", sub_graph)
print("edge_indice : ", edge_indice)
print("loss : ", loss)

final results : 
edge_mask :  tensor([0.0337, 0.0335, 0.0333, 0.0337, 0.9667, 0.9667, 0.0338, 0.9662, 0.0334,
        0.0337, 0.0338, 0.0337, 0.9663, 0.0337, 0.0324, 0.0333, 0.0333, 0.0333,
        0.0338, 0.9663, 0.0335, 0.9674, 0.0338, 0.9662])
sub_graph :  Graph(num_nodes=21, num_edges=24,
      ndata_schemes={'feature': Scheme(shape=(1, 76), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'Edge_indx': Scheme(shape=(), dtype=torch.int64), 'label': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(1, 76), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)})
edge_indice :  5
loss :  0.7548076179623604


In [119]:
indx_edges_to_explain = pd.read_csv('/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Models/Edges_to_explain.csv', encoding="ISO-8859–1", dtype = str)
indx_edges_to_explain = indx_edges_to_explain.apply(pd.to_numeric)
print(indx_edges_to_explain.dtypes)
print(indx_edges_to_explain)

# results_df = pd.DataFrame(columns = ['edge_indx', 'label', 'edge_indice', 'sg', 'edge_mask', 'loss'])
results_df = pd.DataFrame(columns = ['edge_indx', 'label', 'loss'])

Edge_indx    int64
label        int64
dtype: object
      Edge_indx  label
0         98607      0
1        102377      0
2          4448      0
3         39811      0
4         23951      0
...         ...    ...
3119     127231      1
3120      42725      1
3121      84483      1
3122       7892      1
3123      85275      1

[3124 rows x 2 columns]


In [120]:
print("nb edges to explain =", len(indx_edges_to_explain['Edge_indx']))

for i, x in enumerate(indx_edges_to_explain['Edge_indx']):
    if (i % 100) == 0 :
        print(f"{i}th edge")
    edge_indice, sub_graph, edge_mask, loss = explain_edges(model1_test, x, Test_Graph1, node_features_test1, edge_features_test1)
    results_df.loc[-1] = [x, indx_edges_to_explain['label'][i], loss]  # adding a row
    results_df.index = results_df.index + 1  # shifting index

nb edges to explain = 3124
0th edge


NameError: name 'dddddddd' is not defined

In [122]:
results_df.to_csv(f'/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/Models/Final_results.csv', sep=',', index = False)
print(results_df)

   edge_indx  label      loss
0    98607.0    0.0  0.760596
